# Step 1) Data Preparation

In [ ]:
%run data_prep.py INTC

import pandas as pd

df = pd.read_csv("../1_Data/INTC.csv",infer_datetime_format=True, parse_dates=['dt'], index_col=['dt'])

trainCount=int(len(df)*0.4)
dfTrain = df.iloc[:trainCount]

dfTest = df.iloc[trainCount:]
dfTest.to_csv('local_test/test_dir/input/data/training/data.csv')
dfTest.head()

In [ ]:
%matplotlib notebook
dfTest["close"].plot()

# Step 2) Modify Strategy Configuration 

In the following cell, you can adjust the parameters for the strategy.

* `user` = Name for Leaderboard (optional)
* `go_long` = Go Long for Breakout (true or false)
* `go_short` = Go Short for Breakout (true or false)
* `period` = Length of window for previous high and low
* `size` = The number of shares for a transaction

`Tip`: A good starting point for improving the strategy is to lengthen the period of the previous high and low. Equity Markets tend to have a long bias and if you only consider long trades this might improve the performance.

In [ ]:
%%writefile model/algo_config
{ "user" : "user",
  "go_long" : true,
  "go_short" : true,
  "period" : 9,
  "size" : 1000
}

In [ ]:
%run update_config.py daily_breakout

# Step 3) Modify Strategy Code

`Tip`: A good starting point for improving the strategy is to add additional indicators like ATR (Average True Range) before placing a trade. You want to avoid false signals if there is not enough volatility.

Here are some helpful links:
* Backtrader Documentation: https://www.backtrader.com/docu/strategy/
* TA-Lib Indicator Reference: https://www.backtrader.com/docu/talibindautoref/
* Backtrader Indicator Reference: https://www.backtrader.com/docu/indautoref/

In [ ]:
%%writefile model/algo_daily_breakout.py
import backtrader as bt
from algo_base import *
import pytz
from pytz import timezone

class MyStrategy(StrategyTemplate):

    def __init__(self):  # Initiation
        super(MyStrategy, self).__init__()
        self.highest = bt.ind.Highest(period=self.config["period"])
        self.lowest = bt.ind.Lowest(period=self.config["period"])
        self.size = self.config["size"]

    def next(self):  # Processing
        super(MyStrategy, self).next()
        dt=self.datas[0].datetime.datetime(0)
        if not self.position:
            if self.config["go_long"] and self.datas[0] > self.highest[-1]:
                self.buy(size=self.size) # Go long
            elif self.config["go_short"] and self.datas[0] < self.lowest[-1]:
                self.sell(size=self.size) # Go short
        elif self.position.size>0 and self.datas[0] < self.highest[-1]:
            self.close()
        elif self.position.size<0 and self.datas[0] > self.lowest[-1]:          
            self.close()

# Step 4) Backtest Locally (without submitting performance)

**Please note that the initial docker image build may take up to 5 min. Subsequent runs are fast.**

In [ ]:
#Build Local Algo Image
!docker build -t algo_$(cat model/algo_name) .
!docker run -v $(pwd)/local_test/test_dir:/opt/ml --rm algo_$(cat model/algo_name) train

In [ ]:
from IPython.display import Image
Image(filename='local_test/test_dir/model/chart.png')

## Refine your trading strategy (step 2 to 4). Once you are ready to submit the performance of your strategy, move on to the next step.

# Step 5) Run Simulation Backtest on SageMaker and submit performance

**Please note that the backtest in SageMaker runs each time with a new simulated dataset to validate the performance of the strategy. Feel free to run it multiple times.**

In [ ]:
#Deploy Algo Image to ECS
!./build_and_push.sh

In [ ]:
#Run Remote Backtesting via SageMaker
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator 

role = get_execution_role()
sess = sage.Session()

WORK_DIRECTORY = 'local_test/test_dir/input/data/training'
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix='data')
print(data_location)

with open('model/algo_config', 'r') as f:
    config = json.load(f)
algo_name=config['algo_name']

config['sim_data']=True

prefix='algo_'+algo_name
job_name=prefix.replace('_','-')

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{prefix}:latest'

algo = sage.estimator.Estimator(
    image_name=image,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.m4.xlarge',
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess,
    base_job_name=job_name,
    hyperparameters=config,
    metric_definitions=[
        {
            "Name": "algo:pnl",
            "Regex": "Total PnL:(.*?)]"
        },
        {
            "Name": "algo:sharpe_ratio",
            "Regex": "Sharpe Ratio:(.*?),"
        }
    ])
algo.fit(data_location)

In [ ]:
#Get Algo Metrics
from sagemaker.analytics import TrainingJobAnalytics

latest_job_name = algo.latest_training_job.job_name
metrics_dataframe = TrainingJobAnalytics(training_job_name=latest_job_name).dataframe()
metrics_dataframe

In [ ]:
#Get Algo Chart from S3
model_name=algo.model_data.replace('s3://'+sess.default_bucket()+'/','')
import boto3
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(sess.default_bucket())
my_bucket.download_file(model_name,'model.tar.gz')
!tar -xzf model.tar.gz
!rm model.tar.gz
from IPython.display import Image
Image(filename='chart.png') 

### Congratulations! You've completed this strategy. Verify your submission on the leaderboard.

In [ ]:
%run leaderboard.py